Надо достать:
- для фильмов:
    * Название,
    * Дату выхода,
    * Слоган,
    * Бюджет,
    * Выручку,
    * Продолжительность,
    * Рейтинг,
    * Описание,
    * Жанры,
    * Страны,
    * Персонал: id человека, его роль при создании фильма, исполняемый персонаж при наличии.
- для сериалов:
    * Название,
    * Дату выхода,
    * Слоган,
    * Бюджет,
    * Рейтинг,
    * Описание,
    * Жанры,
    * Страны,
    * Персонал.
    * Для каждого эпизода:
        * Номер сезона,
        * Номер эпизода в сезоне,
        * Название,
        * Дата выхода,
        * Продолжительность,
        * Рейтинг.
- Для персонала:
    * Имя,
    * Дату рождения,
    * Дату смерти (при наличии),
    * Место рождения,
    * Описание.

# Start

In [1]:
import pandas as pd
import imdb
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
ia = imdb.IMDb()

In [3]:
file_path = 'data.csv'
df = pd.read_csv(file_path)

# Parsing via IMDb

### Movies and TV Shows

In [4]:
movies_data = []
personnel_data = {}
personnel_ids = set()

In [5]:
roles = {'cast', 'director', 'writer', 'producer', 'composer', 'cinematographer', 'editor', 'editorial department',
         'casting director', 'production design', 'art direction', 'set decoration', 'costume designer', 'make up',
         'production manager', 'assistant director', 'art department', 'sound crew', 'special effects',
         'visual effects', 'stunt performer', 'camera and electrical department', 'casting department',
         'costume department', 'location management', 'music department', 'script department',
         'transportation department', 'miscellaneous crew', 'thanks'}
genres = set()
countries = set()

In [6]:
def fetch_movie_data(movie_id: str, fetch_personnel=False, cnt=0, attempt_limit=5) -> dict:
    if cnt >= attempt_limit:
        return {}

    global roles, personnel_ids, ia, personnel_data, genres, countries

    try:
        movie = ia.get_movie(movie_id)
        ia.update(movie, info=['main', 'plot', 'taglines'])
    except Exception as e:
        print(f"ОШИБКА ({cnt}/{attempt_limit}) при обработке movie_id {movie_id}:\n{e}\n" + '=' * 80)
        return fetch_movie_data(movie_id, cnt + 1)

    local_genres = movie.get('genres', [])
    local_countries = movie.get('countries', [])
    movie_data = {
        'MovieID': movie_id,
        'Title': movie.get('title'),
        'Release Date': movie.get('original air date') or movie.get('year'),
        'Tagline': movie.get('taglines')[0] if movie.get('taglines') else None,
        'Budget': movie.get('box office', {}).get('Budget'),
        'Revenue': movie.get('box office', {}).get('Cumulative Worldwide Gross'),
        'Duration': movie.get('runtimes')[0] if movie.get('runtimes') else None,
        'Rating': movie.get('rating'),
        'Description': movie.get('plot outline'),
        'Genres': local_genres,
        'Countries': local_countries,
    }
    genres |= set(local_genres)
    countries |= set(local_countries)

    for role in roles:
        movie_data[role] = {} if role == 'cast' else []
        for person in movie.get(role, []):
            if isinstance(person, imdb.Person.Person) and person.personID is not None:
                personnel_ids.add(person.personID)
                if fetch_personnel:
                    personnel_data[person.personID] = {
                        'PersonID': person.personID,
                        'Name': person.get('name'),
                        'Birth Date': None,
                        'Death Date': None,
                        'Place of Birth': None,
                        'Biography': None,
                    }
                if role == 'cast':
                    try:
                        movie_data[role][person.personID] = str(person.currentRole)
                    except Exception as e:
                        print(f"ОШИБКА при обработке PersonID {person.personID} ({person}) в фильме {movie_id} ({movie}):\n"
                              f"{e}\n" + '=' * 80)
                else:
                    movie_data[role].append(person.personID)

    return movie_data

In [7]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    movie_id = row['Const'][2:]  # Убираем префикс 'tt'
    movie_data = fetch_movie_data(movie_id, fetch_personnel=True)
    movies_data.append(movie_data)

 52%|█████▏    | 274/525 [33:30<29:10,  6.97s/it]  2025-04-17 02:08:55,957 CRITICAL [imdbpy] /Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0438097/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/url

ОШИБКА (0/5) при обработке movie_id 0438097:
{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0438097/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}


 53%|█████▎    | 277/525 [50:30<10:34:56, 153.62s/it]2025-04-17 02:25:18,312 CRITICAL [imdbpy] /Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt3416828/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/u

ОШИБКА (0/5) при обработке movie_id 3416828:
{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt3416828/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}


100%|██████████| 525/525 [1:32:55<00:00, 10.62s/it]    


In [13]:
len(movies_data)

525

## Personnel

In [75]:
def fetch_person_data(person_id: str, cnt: int=0, attempt_limit=5) -> (str, dict):
    # Gets only names but takes a lot of time to do it
    if cnt >= attempt_limit:
        return {}

    global ia

    try:
        person = ia.get_person(person_id)
        ia.update(person, info=['main', 'biography'])
    except Exception as e:
        print(f"ОШИБКА ({cnt}/{attempt_limit}) при обработке PersonID {person_id}:\n{e}\n" + '=' * 80)
        return fetch_person_data(person_id, cnt + 1)

    return person_id, {
        'PersonID': person_id,
        'Name': person.get('name'),
        'Birth Date': person.get('birth date'),
        'Death Date': person.get('death date'),
        'Place of Birth': person.get('birth place'),
        'Biography': person.get('mini biography')[0] if person.get('mini biography') else None
    }

In [76]:
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(fetch_person_data, person_id) for person_id in personnel_ids]

    for future in tqdm(as_completed(futures), total=len(futures), desc="Получение данных о персонале"):
        result = future.result()
        if result:
            personnel_data[result[0]] = result[1]

Получение данных о персонале:  67%|██████▋   | 924/1384 [26:48<8:46:50, 68.72s/it]  2025-04-10 19:58:20,259 CRITICAL [imdbpy] /Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm5978942/', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
               ^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/urllib/request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/

ОШИБКА (0/5) при обработке PersonID 5978942:
{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm5978942/', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}


2025-04-10 19:58:21,647 CRITICAL [imdbpy] /Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm7917848/fullcredits', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/Users/arif_meilanov/Desktop/Arif/Projects/Python/DataBases/.venv/lib/python3.11/site-packages/imdb/parser/http/__init__.py", line 234, in retrieve_unicode
    content = response.read()
              ^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 460, in read
    return self._read_chunked(amt)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 592, in _read_chunked
    value.append(self.

ОШИБКА (0/5) при обработке PersonID 7917848:
{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm7917848/fullcredits', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
ОШИБКА (0/5) при обработке PersonID 6568870:
{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm6568870/bio', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
ОШИБКА (0/5) при обработке PersonID 1344937:
{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm1344937/fullcredits', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
ОШИБКА (0/5) при обработке PersonID 0252922:
{'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/name/nm0252922/fullcredits', 'proxy': '', 'exception type': 'IOError', 'original exception': URLError(TimeoutError('_ssl.c:985: The handshake operation

Получение данных о персонале: 100%|██████████| 1384/1384 [35:37<00:00,  1.54s/it] 


In [94]:
personnel_data

{'0000995': {'PersonID': '0000995',
  'Name': 'Ellen Burstyn',
  'Birth Date': None,
  'Death Date': None,
  'Place of Birth': None,
  'Biography': None},
 '0000190': {'PersonID': '0000190',
  'Name': 'Matthew McConaughey',
  'Birth Date': None,
  'Death Date': None,
  'Place of Birth': None,
  'Biography': None},
 '3237775': {'PersonID': '3237775',
  'Name': 'Mackenzie Foy',
  'Birth Date': None,
  'Death Date': None,
  'Place of Birth': None,
  'Biography': None},
 '0001475': {'PersonID': '0001475',
  'Name': 'John Lithgow',
  'Birth Date': None,
  'Death Date': None,
  'Place of Birth': None,
  'Biography': None},
 '3154303': {'PersonID': '3154303',
  'Name': 'Timothée Chalamet',
  'Birth Date': None,
  'Death Date': None,
  'Place of Birth': None,
  'Biography': None},
 '0654648': {'PersonID': '0654648',
  'Name': 'David Oyelowo',
  'Birth Date': None,
  'Death Date': None,
  'Place of Birth': None,
  'Biography': None},
 '2180792': {'PersonID': '2180792',
  'Name': 'Collette Wolfe

# Collecting all information into dataframes

In [8]:
movies_df = pd.DataFrame(movies_data).set_index('MovieID')
movies_df.to_csv('movies_data.csv')

personnel_df = pd.DataFrame(personnel_data.values()).set_index('PersonID')
personnel_df.to_csv('personnel_data.csv')

genres_df = pd.DataFrame({'Genre': list(genres)})
genres_df.to_csv('genres_data.csv', index=False)

countries_df = pd.DataFrame({'Country': list(countries)})
countries_df.to_csv('countries_data.csv', index=False)

## Check

In [9]:
pd.read_csv('movies_data.csv', index_col=0).head()

,Title,Release Date,Tagline,Budget,Revenue,Duration,Rating,Description,Genres,Countries,...,costume department,script department,director,editorial department,casting director,cast,sound crew,editor,casting department,set decoration
MovieID,,,,,,,,,,,,,,,,,,,,,
816692,Interstellar,06 Nov 2014 (Russia),Mankind was born on Earth. It was never meant ...,"$165,000,000 (estimated)","$675,020,017",169.0,8.7,In the near future around the American Midwest...,"['Adventure', 'Drama', 'Sci-Fi']","['United States', 'United Kingdom', 'Canada']",...,"['3335867', '2467968', '0070968', '0097003', '...","['4836850', '0311826']",['0634240'],"['1333123', '4975838', '1959859', '1519080', '...",['0660667'],"{'0000995': 'Murph', '0000190': 'Cooper', '323...","['0044847', '0090635', '0193503', '0221262', '...",['0809059'],"['1598836', '5738223', '0364456', '5751780', '...","['0275361', '0469002']"
5613056,Piper,16 Jun 2016 (Russia),NaN,NaN,NaN,6.0,8.3,A young and bright-eyed Sandpiper is motivated...,"['Animation', 'Short', 'Family']",['United States'],...,[],[],['0054659'],"['1629055', '1649258', '3041455', '3050121', '...",[],{},"['0460274', '0499761', '0809950', '0193503']",['2204583'],[],[]
103639,Aladdin,02 Apr 1994 (Russia),Wish granted! (DVD re-release),"$28,000,000 (estimated)","$504,050,219",90.0,8.0,Aladdin is a poor street urchin who spends his...,"['Animation', 'Adventure', 'Comedy', 'Family',...",['United States'],...,[],[],"['0166256', '0615780']","['0024305', '1072591', '0151638', '1072722', '...",['0970331'],"{'0918334': 'Aladdin', '0000245': 'Genie / Ped...","['0056199', '0058897', '0085669', '0172229', '...",['0677177'],['3258501'],[]
398286,Tangled,25 Nov 2010 (Russia),They're taking adventure to new lengths.,"$260,000,000 (estimated)","$591,794,936",100.0,7.7,After receiving the healing powers from a magi...,"['Animation', 'Adventure', 'Comedy', 'Family',...","['United States', 'Japan']",...,[],[],"['1977355', '0397174']","['0046721', '0111448', '0163087', '3516524', '...",['0816924'],"{'0601553': 'Rapunzel', '1157048': 'Flynn Ride...","['0045344', '0124886', '1103865', '0003949', '...",['0581580'],"['0235261', '2064023']",[]
120762,Mulan,25 Dec 1998 (Russia),An Epic Motion Picture Event,"$90,000,000 (estimated)","$304,320,254",87.0,7.7,A young female decides to fill in for her fami...,"['Animation', 'Adventure', 'Comedy', 'Family',...",['United States'],...,['2145472'],[],"['0051643', '0176905']","['0046721', '1125323', '0099263', '0134945', '...",['0483294'],"{'0001208': 'Shan-Yu', '0001213': 'Yao', '0791...","['0070489', '0100806', '0115177', '0138939', '...",['0446668'],"['0006746', '0002888', '0192926', '1125923']",[]


In [10]:
pd.read_csv('personnel_data.csv', index_col=0).head()

,Name,Birth Date,Death Date,Place of Birth,Biography
PersonID,,,,,
1743866,Craig Abele,NaN,NaN,NaN,NaN
3167648,Lauren Abiouness,NaN,NaN,NaN,NaN
2354038,Cheryl Allsen,NaN,NaN,NaN,NaN
46517,Laura Bagano,NaN,NaN,NaN,NaN
58921,Pierre Bartlette,NaN,NaN,NaN,NaN


In [11]:
pd.read_csv('genres_data.csv').head()

,Genre
0,Mystery
1,Sci-Fi
2,Music
3,Sport
4,Adventure


In [12]:
pd.read_csv('countries_data.csv').head()

,Country
0,Gambia
1,Thailand
2,Qatar
3,Australia
4,Russia


In [12]:
movies_df

,Title,Release Date,Tagline,Budget,Revenue,Duration,Rating,Description,Genres,Countries,...,stunt performer,camera and electrical department,casting department,costume department,location management,music department,script department,transportation department,miscellaneous crew,thanks
MovieID,,,,,,,,,,,,,,,,,,,,,
0816692,Interstellar,06 Nov 2014 (Russia),Mankind was born on Earth. It was never meant ...,"$165,000,000 (estimated)","$675,020,017",169,8.7,In the near future around the American Midwest...,"[Adventure, Drama, Sci-Fi]","[United States, United Kingdom, Canada]",...,"[0008619, 1640149, 0496726, 0183037, 1259016, ...","[5107911, 0021773, 5528862, 4867707, 0042078, ...","[1598836, 5738223, 0364456, 5751780, 0516950, ...","[3335867, 2467968, 0070968, 0097003, 2782734, ...","[2261266, 0115276, 2422357, 2279296, 1562095, ...","[6776035, 7089563, 5214176, 0033575, 9892076, ...","[4836850, 0311826]","[1567468, 5108970, 0129030, 2281713, 6503377, ...","[1561964, 0012515, 0022945, 0024552, 1153895, ...","[3394005, 6568891, 7920311, 0122741, 9953469, ..."
0816692,Interstellar,06 Nov 2014 (Russia),Mankind was born on Earth. It was never meant ...,"$165,000,000 (estimated)","$675,020,017",169,8.7,In the near future around the American Midwest...,"[Adventure, Drama, Sci-Fi]","[United States, United Kingdom, Canada]",...,"[0008619, 1640149, 0496726, 0183037, 1259016, ...","[5107911, 0021773, 5528862, 4867707, 0042078, ...","[1598836, 5738223, 0364456, 5751780, 0516950, ...","[3335867, 2467968, 0070968, 0097003, 2782734, ...","[2261266, 0115276, 2422357, 2279296, 1562095, ...","[6776035, 7089563, 5214176, 0033575, 9892076, ...","[4836850, 0311826]","[1567468, 5108970, 0129030, 2281713, 6503377, ...","[1561964, 0012515, 0022945, 0024552, 1153895, ...","[3394005, 6568891, 7920311, 0122741, 9953469, ..."
5613056,Piper,16 Jun 2016 (Russia),None,None,None,6,8.3,A young and bright-eyed Sandpiper is motivated...,"[Animation, Short, Family]",[United States],...,[],[0930430],[],[],[],"[1725223, 2117278, 0230392, 5912967, 2895026, ...",[],[],"[11038533, 6606843, 2584753, 4598381, 1645100,...","[1443869, 0129269, 0132357, 0146216, 1095824, ..."
0103639,Aladdin,02 Apr 1994 (Russia),Wish granted! (DVD re-release),"$28,000,000 (estimated)","$504,050,219",90,8.0,Aladdin is a poor street urchin who spends his...,"[Animation, Adventure, Comedy, Family, Fantasy...",[United States],...,[],"[0007426, 0971741, 0971779, 3238989, 1074770, ...",[3258501],[],[],"[0039141, 0071858, 0098567, 0152940, 0268520, ...",[],[],"[3117930, 0017597, 0019844, 3128473, 9654276, ...",[0000048]
0398286,Tangled,25 Nov 2010 (Russia),They're taking adventure to new lengths.,"$260,000,000 (estimated)","$591,794,936",100,7.7,After receiving the healing powers from a magi...,"[Animation, Adventure, Comedy, Family, Fantasy...","[United States, Japan]",...,[],"[0095675, 0440773, 0819777, 1306668, 0972165]","[0235261, 2064023]",[],[0089077],"[1791092, 0052640, 2924361, 0074478, 1259350, ...",[],[],"[2143013, 3719647, 6083921, 3724321, 2493025, ...","[0015935, 14442431, 2346705, 3719560, 2632553,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197624,Law Abiding Citizen,05 Nov 2009 (Russia),The System Must Pay,"$50,000,000 (estimated)","$126,690,726, 12 Aug 2012",109,7.4,Clyde Shelton's family is brutally murdered. T...,"[Action, Crime, Drama, Thriller]",[United States],...,"[2099236, 1347896, 0025574, 0046966, 0076776, ...","[1038910, 0038435, 3176090, 0046360, 0070484, ...","[3130459, 0373519, 3069286, 1090456, 3461704, ...","[3738679, 2011201, 3381844, 0321834, 1450201, ...","[2978447, 2405412, 1776344, 2094930, 0331055, ...","[0962832, 1423589, 0253402, 0343065, 2087283, ...","[0076689, 0283957, 0001104, 1471001, 1417242]","[3848547, 2176940, 3443720, 3848086, 1769203, ...","[3187348, 0032151, 0047358, 0053815, 3095216, ...","[1061443, 2995144, 0873531]"
9054904,Why Women Kill,2019,1 House. 3 Decades. 3 Women. Same Problem.,None,None,